In [3]:
import numpy as np
import pandas as pd
import googlemaps
import json


charities_raw = pd.read_csv('Charities_BC_PoveryCommunity.csv')
partners_raw = pd.read_csv('UWBC_PartnerOrganizations.csv')

charities_filtered = charities_raw.filter(items=['Legal name','Mailing address','City','Province','Postal code','Country'])
partners_filtered = partners_raw.filter(items=['NAME1','Addressline1','CITY','postalcode'])

# Strip whitespace
for column in partners_filtered.columns:
    partners_filtered[column] = partners_filtered[column].str.strip()

charities_filtered['full_address'] = charities_filtered[['Mailing address','City','Province','Postal code','Country']].agg(', '.join,axis=1)
partners_filtered['full_address'] = partners_filtered[['Addressline1','CITY','postalcode']].agg(', '.join,axis=1)

charities_latlong = charities_filtered.loc[:,['Legal name','full_address']]
partners_latlong = partners_filtered.loc[:,['NAME1','full_address']]

charities_filtered.to_csv('charity_addresses.csv',index=False)

charities_latlong[['Lat','Long']] = 0
partners_latlong[['Lat','Long']] = 0

charities_test = charities_latlong.head(5)
#print(charities_test)
print(partners_latlong.shape[0])

gmaps = googlemaps.Client(key='<YOUR-API-KEY>')

for i in range(charities_latlong.shape[0]):
    try:
        loc_json = gmaps.geocode(charities_latlong.at[i,'full_address'])
        lat = loc_json[0]['geometry']['location']['lat']
        long = loc_json[0]['geometry']['location']['lng']
        charities_latlong.at[i,'Lat'], charities_latlong.at[i,'Long'] = lat, long
    except IndexError as e:
        print(e)

charities_latlong.to_csv('charities_latlong.csv', index=False)

for j in range(partners_latlong.shape[0]):
    try:
        loc_json = gmaps.geocode(partners_latlong.at[j,'full_address'])
        lat = loc_json[0]['geometry']['location']['lat']
        long = loc_json[0]['geometry']['location']['lng']
        partners_latlong.at[j,'Lat'], partners_latlong.at[j,'Long'] = lat, long
    except IndexError as e:
        print(e)

partners_latlong.to_csv('partners_latlong.csv',index=False)



0    3245 THIRD AVENUE, PORT ALBERNI, BC, V9Y4C9, CA
1                   PO BOX 698, HOPE, BC, V0X1L0, CA
2           901 KINGS ROAD, VICTORIA, BC, V8T1W5, CA
3         1095 SHUSWAP AVE, SICAMOUS, BC, V0E2V1, CA
4    227 6TH STREET, NEW WESTMINSTER, BC, V3L3A5, CA
Name: full_address, dtype: object
0             201 E 23 Street, North Vancouver, V7L3E4
1                   4567 Heather St, Vancouver, V5Z0C9
2           200 - 678 E Hastings St, Vancouver, V6A1R1
3                            PO Box 25, Sardis, V2R1A5
4    501 - 104  1015 Columbia St, New Westminster, ...
Name: full_address, dtype: object
633
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
